In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import random
import sklearn
import tslearn

from tslearn.clustering import KernelKMeans
from tslearn.utils import to_time_series_dataset
import numpy as np
import networkx as nx

import sys
sys.path.append("../")

from pyspark.sql import SparkSession
from pyspark.sql.functions import format_string
from sklearn.manifold import TSNE
from analysis.spark import SparkEngine
from analysis.preprocess import extract_states

spark = SparkSession.builder.getOrCreate()
spark_engine = SparkEngine(spark)

ModuleNotFoundError: No module named 'tslearn'

In [ ]:
import altair as alt
from vega_datasets import data
alt.data_transformers.disable_max_rows()


In [ ]:
# If you already uploaded data to HFS
filepath = "/user/gw2145/integrated.csv"
spark = SparkSession.builder.getOrCreate()
covid_data = spark.read.format('csv').options(header='true', inferschema='true').load(filepath)
covid_data.createOrReplaceTempView("covid")
integrated_src_data = covid_data.toPandas()
integrated_data = integrated_src_data.fillna(0)
states = extract_states(integrated_data)
states

In [ ]:
# if you did not upload data to HFS
local_path = "./integrated_data.csv"
covid_pd = pd.read_csv(local_path)
covid_df = spark.createDataFrame(covid_pd)
covid_data = covid_df
covid_data.createOrReplaceTempView("covid")
integrated_src_data = covid_data.toPandas()
integrated_data = integrated_src_data.fillna(0)
states = extract_states(integrated_data)
states

In [ ]:
# Data Analysis



In [ ]:
# Cluster state data by time series k-means


In [ ]:
from analysis.time_series import get_state_embeddings, add_cluster_info, extract_state_series, cluster_states_by_ts
# This cell hold the functions for layout

In [ ]:
X = extract_state_series(integrated_data)
labels = cluster_states_by_ts(X)


In [ ]:
embeddings = get_state_embeddings(states, labels)
integrated_data = add_cluster_info(integrated_data, embeddings)

In [ ]:
from analysis.visualization import plot_ts_clusters

plot_ts_clusters(integrated_data, embeddings)

In [ ]:
from analysis.preparation import load_state_info
state2fips_df, populations = load_state_info()

In [ ]:
from analysis.visualization import plot_map

plot_map(integrated_data, state2fips_df, populations, date="2021-01-31", attr="cases_by_population")

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Load training data


In [ ]:
import pyspark.sql.functions as F

df = spark_engine.join_population(covid_data, populations)
df1 = spark_engine.add_idx(df)


In [ ]:
mobility_metrics = [
    "retail_and_recreation_percent_change_from_baseline",
    "grocery_and_pharmacy_percent_change_from_baseline",
    "workplaces_percent_change_from_baseline",
    "residential_percent_change_from_baseline"
]
state_metric_correlation = spark_engine.generate_state_metric_corr(df1, states)
state_metric_correlation

In [ ]:
import importlib
import analysis.visualization
importlib.reload(analysis.visualization)
from analysis.visualization import plot_state_metric_corr, plot_corr_map
import warnings
warnings.filterwarnings('ignore')
source_corr_matrix = []
        
plot_state_metric_corr(state_metric_correlation, mobility_metrics)
# state_metric_correlation_log

In [ ]:
plot_corr_map(state_metric_correlation, state2fips_df, mobility_metrics)

In [ ]:
df.createOrReplaceTempView("analysis_data")

In [ ]:
from datetime import datetime, timedelta




In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install nltk
# !pip install nl4dv

import importlib
import st_nli
importlib.reload(st_nli)
from st_nli import NLInterface


query_sent = "Show the case rate in New England"
query_sent = "Show the case rate on 01/31/2021 in New England"
query_sent = input()
interface = NLInterface()
interface.run(query_sent, df)
